## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [2]:
with open("../input/tps-may-data-preprocess-v4/TPS_May_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

del processed_data
gc.collect()

0

In [3]:
cat_cols = train_df.iloc[:,0:50].columns
train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [4]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (99918, 372) 
Ytrain: (99918,) 
Ytrain_oh: (99918, 4) 
Xtest: (50000, 372)


20

## Build and validate the model

In [5]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(2021)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 4))
y_pred_final_cb = np.zeros((Xtest.shape[0], 4))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
        val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            max_ctr_complexity=15,
            od_wait=1000, 
            od_type='Iter',
            use_best_model=True,
            bootstrap_type='Poisson',
            learning_rate=0.01465,
            reg_lambda=0.01864,
            subsample=0.68135,
            max_depth=6, 
            min_data_in_leaf=1,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=200)

        y_pred = model.predict_proba(val_x)
        y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_cb[val] += y_pred
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(NUM_SEED)
y_pred_final_cb = y_pred_final_cb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 1.3775360	test: 1.3775502	best: 1.3775502 (0)	total: 25.2ms	remaining: 2m 6s
200:	learn: 1.0989227	test: 1.1054170	best: 1.1054170 (200)	total: 3.19s	remaining: 1m 16s
400:	learn: 1.0869820	test: 1.0990816	best: 1.0990816 (400)	total: 6.18s	remaining: 1m 10s
600:	learn: 1.0789813	test: 1.0965777	best: 1.0965777 (600)	total: 9.42s	remaining: 1m 8s
800:	learn: 1.0721895	test: 1.0950668	best: 1.0950653 (799)	total: 12.3s	remaining: 1m 4s
1000:	learn: 1.0659321	test: 1.0942354	best: 1.0942354 (1000)	total: 16.1s	remaining: 1m 4s
1200:	learn: 1.0599710	test: 1.0936488	best: 1.0936401 (1191)	total: 19.2s	remaining: 1m
1400:	learn: 1.0544019	test: 1.0933612	best: 1.0933612 (1400)	total: 22.1s	remaining: 56.7s
1600:	learn: 1.0489724	test: 1.0932290	best: 1.0931877 (1524)	total: 25s	remaining: 53s
bestTest = 1.093187734
bestIteration = 1524
Shrink model to first 1525 iterations.
Seed-85 | Fold-0 | OOF Score: 1.093187951494105
0:	learn: 1.3775406	test: 1.3775273	best: 1.3775273 (0)	tot

In [6]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

## Create submission file

In [7]:
y_pred_final_cb = np.clip(y_pred_final_cb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_cb[:,0]
submit_df['Class_2'] = y_pred_final_cb[:,1]
submit_df['Class_3'] = y_pred_final_cb[:,2]
submit_df['Class_4'] = y_pred_final_cb[:,3]
submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.093487,0.613125,0.174282,0.119105
1,100001,0.078744,0.668017,0.153557,0.099682
2,100002,0.083619,0.627218,0.176709,0.112454
3,100003,0.089981,0.536541,0.269020,0.104458
4,100004,0.075227,0.614826,0.192633,0.117314


In [8]:
submit_df.to_csv("./CB_submission.csv", index=False)